In [1]:
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import sys
from selenium.webdriver.common.by import By
import pandas as pd
url1 = "https://www.youtube.com/"

# To block the notifications
options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\shraDDhar\Downloads\chromedriver.exe')
driver.get(url1)
time.sleep(10)
p= driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')
p.send_keys('kitchen technology')
#search-icon-legacy
po= driver.find_element(By.CSS_SELECTOR,'#search-icon-legacy')
po.click()
df=pd.DataFrame()
for i in range(10):
    pri= driver.find_elements(By.CSS_SELECTOR,'#video-title')
    prid=[elem.get_attribute('href') for elem in pri]
    time.sleep(5)
    df.append(prid)
df=df.T

C:\Users\shraDDhar\AppData\Local\Temp\ipykernel_78828\2071549875.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\shraDDhar\Downloads\chromedriver.exe')
C:\Users\shraDDhar\AppData\Local\Temp\ipykernel_78828\2071549875.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(prid)
C:\Users\shraDDhar\AppData\Local\Temp\ipykernel_78828\2071549875.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(prid)
C:\Users\shraDDhar\AppData\Local\Temp\ipykernel_78828\2071549875.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(prid)


In [2]:
prid=[i for i in prid if i!=None]

In [3]:
prid

['https://www.youtube.com/watch?v=XnUTXfd46pg',
 'https://www.youtube.com/watch?v=JVeMgvTsBVM',
 'https://www.youtube.com/watch?v=NzSl631j0F8',
 'https://www.youtube.com/watch?v=BqbC-TaXjOg']

In [ ]:
from youtube_comment_scraper_python import *
pop=pd.DataFrame()
for i in prid:
    youtube.open(i)
    response=youtube.video_comments()
    data=response['body']
    g=pd.DataFrame(data)
    pop=pd.concat([pop,g])

In [15]:
pop

""


In [6]:
p=pd.DataFrame(pop[0])

In [7]:
p

,Comment,Likes,Time,user
0,Amei parabéns,2,2 years ago,
1,"For those engineer designers watching, I want ...",4,1 year ago,
2,who is the vender with the extendable table?,1,1 year ago,


In [8]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\shraDDhar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
sid = SentimentIntensityAnalyzer()
p['scores'] =p['Comment'].apply(lambda review: sid.polarity_scores(review))
p['compound']  = p['scores'].apply(lambda score_dict: score_dict['compound'])

In [10]:
p

,Comment,Likes,Time,user,scores,compound
0,Amei parabéns,2,2 years ago,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,"For those engineer designers watching, I want ...",4,1 year ago,,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.2732
2,who is the vender with the extendable table?,1,1 year ago,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(p['Comment'].values.astype('U'))
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [12]:
first_topic = LDA.components_[0]

NameError: name 'LDA' is not defined

In [23]:
top_topic_words = first_topic.argsort()[-10:]

In [24]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

teflon
use
used
safe
thank
cooking
time
video
ceramic
cookware


In [25]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['teflon', 'use', 'used', 'safe', 'thank', 'cooking', 'time', 'video', 'ceramic', 'cookware']


Top 10 words for topic #1:
['stick', 'glass', 'safe', 'stainless', 'ceramic', 'grade', 'food', 'aluminum', 'iron', 'cast']


Top 10 words for topic #2:
['harmful', 'just', 'away', 'like', 'pan', 'safe', 'video', 'steel', 'glass', 'food']


Top 10 words for topic #3:
['best', 'cooking', 'pans', 'll', 'food', 'health', 'thank', 'using', 'foil', 'harmful']


Top 10 words for topic #4:
['thank', 'foil', 'harmful', 'think', 'just', 'saw', 'health', 'safe', 'copper', 'buy']




In [26]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(20, 5)

In [27]:
p['Topic'] = topic_values.argmax(axis=1)

In [28]:
p

,Comment,Likes,Time,UserLink,user,scores,compound,Topic
0,TIMESTAMPSTeflon cookware 0:38Aluminum cookwar...,1.1K,4 years ago,https://www.youtube.com/channel/UC4rlAVgAK0SGk...,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0
1,"After watching this video, you'll be consideri...",4.1K,2 years ago,NaN,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0
2,My great grandma used cast iron her whole life...,126,1 year ago,NaN,,"{'neg': 0.0, 'neu': 0.832, 'pos': 0.168, 'comp...",0.8442,2
3,Safe:-cast iron or enamel coated cast iron-gla...,774,1 year ago (edited),NaN,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,1
4,I’ll stick with my cast iron pans inherited fr...,578,10 months ago,NaN,,"{'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'comp...",0.6369,1
5,After seeing this I have started cooking direc...,97,1 year ago (edited),NaN,,"{'neg': 0.118, 'neu': 0.882, 'pos': 0.0, 'comp...",-0.3400,0
6,I am using 100% ceramic cookware...the best in...,2,1 month ago (edited),NaN,,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp...",0.9118,0
7,"SO basically Cast Iron, Glass, Food Grade Stai...",9.3K,4 years ago,NaN,,"{'neg': 0.115, 'neu': 0.609, 'pos': 0.277, 'co...",0.6739,1
8,"Thank you ,here in Brazil you can easily buy c...",58,2 years ago,NaN,,"{'neg': 0.0, 'neu': 0.716, 'pos': 0.284, 'comp...",0.8779,0
9,I'm so glad I saw this video. I've been using ...,15,7 months ago,NaN,,"{'neg': 0.0, 'neu': 0.687, 'pos': 0.313, 'comp...",0.8610,2
